File: ratings.ipynb\
Author: alexkobz\
Date: 07.05.2024\
Task: DSRFU-275

#### Шаг 0. Импорт и получение токена

In [1]:
from datetime import datetime as dt, date as d, timedelta
import pandas as pd
import asyncio
import nest_asyncio; nest_asyncio.apply()
import numpy as np

# последний день месяца
REPORT_DATE = pd.to_datetime(d.today() - timedelta(days=14) + pd.offsets.MonthEnd(n=1)).strftime("%Y-%m-%d")
# увеличиваем количество столбцов и строк для просмотра
pd.set_option('display.max_rows', 200, 'display.max_columns', 200)

In [2]:
from rudata.RuDataDF import RuDataDF
from rudata.RuDataRequest import RuDataRequest
RuDataRequest.set_headers()

Getting token started
Getting token finished


#### Шаг 1. Загрузка stage таблиц 

[FintoolReferenceData](https://docs.efir-net.ru/dh2/#/Info/FintoolReferenceData?id=post-fintoolreferencedata) - Получить расширенный справочник по финансовым инструментам.

In [3]:
FintoolReferenceData = RuDataDF("FintoolReferenceData").df
FintoolReferenceData.head()

FintoolReferenceData started
post start
post start
post start
post start
post start
post processing
post processing
sleep 1
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
1500, 0
post start
post start
post start
post start
post start
post processing
post processing
sleep 1
post processing
sleep 1
post processing
post processing
sleep 1
sleep 1
sleep 1
1500, 1500
post start
post start
post start
post start
post start
post processing
post processing
sleep 1
sleep 1
post processing
post processing
post processing
sleep 1
sleep 1
sleep 1
1500, 3000
post start
post start
post start
post start
post start
post processing
post processing
sleep 1
post processing
sleep 1
sleep 1
post processing
sleep 1
post processing
sleep 1
1500, 4500
post start
post start
post start
post start
post start
post processing
sleep 1
post processing
post processing
sleep 1
sleep 1
post processing
post processing
sleep 1
sleep 1
1500, 6000
post start
post start
post start
post start


,facevalue_m,faceftname_m,sumissuevol_m,summarketvol_m,numcoupons_m,nrdcode,regcode_m,regdate_m,regdistdate_m,begdistdate_m,enddistdate_m,endmtydate_m,daysall_m,fullname_nrd,rpregorg_nrd,rpstate_nrd,issuenumber,currentfacevalue_nrd,isqualified_nrd,bondseries,islombardcbr_nrd,numcoupons_nrd,facevalue_nrd,faceftname_nrd,summarketvol_nrd,regcode_nrd,regdate_nrd,regorg_nrd,begdistdate_nrd,daysall_nrd,enddistdate_nrd,endmtydate_nrd,regdistdate_nrd,sumissuevol_nrd,drqty,isincodebase_nrd,nrdcompcode,shqty,summarketvolorig_nrd,issuername_nrd,planmtydate_nrd,firstcoupondate_nrd,depositary_nrd,registrar_nrd,placeprice_nrd,micexlist_nrd,minlot_nrd,formarketbonds_nrd,registraracctype_nrd,registraracctypedate_nrd,facialacc_nrd,acc_open_date_nrd,fraction_nrd,specialized_depo_nrd,cmp_code_nsd_issuer_nrd,cmp_code_nsd_managing_nrd,ratetypenamerus_nrd,ratetypenameeng_nrd,coupontypename_nrd,base_month,base_year,basis_nrd,sectypenamerus_nrd,sectypenameeng_nrd,secformnamerus_nrd,secformnameeng_nrd,sectypenamebr_nrd,secstaterus_nrd,secstateeng_nrd,placementtype_nrd,share_category_name_nrd,share_category_name_en_nrd,dr_category_name_nrd,dr_category_name_en_nrd,company_state_name_nrd,rate_type_id,cp_type_id,coupon_period_base_id,sec_type_id,sec_form_id,sec_state_id,placement_type_id,share_category_id,dr_category_mn,company_state_mn,sector4212u,sna2008,fullname_en_nrd,seniorityname,fintoolid,moex_code,moex_name,fullname,isin144a,isincode,nickname,progfintoolid,regcode,regdate,regorg,amortisedmty,basis,bondstructuralpar,cfi,coupontype,fintooltype,guaranteeamount,guaranteetype,haveoffer,isconvertible,isguaranteed,maturitygroup,securitization,accruedintcalctype,country,couponperyear,faceftname,facevalue,guarantval,havedefault,numcoupons,status,sumissueval,sumissuevol,summarketval,summarketvol,borrowername,borrowerokpo,borrowersector,borroweruid,borrowerinn,borrowercountry,borrowerregionname,issuername,issuerokpo,issuersector,issueruid,issuerinn,issuercountry,issuerregionname,assetssector,numguarantors,begdistdate,begmtydate,daysall,enddistdate,endmtydate,privatedist,regdistdate,securitytype,securitykind,nominaltype,update_time,isregion,nsd_service,raterevisionsperyear,floatratename,issubordinated,note,listing_level,havecovenant,haverepayment,haveindexedfv,numdecimals,couponperiod,ismatched,firstcoupondate,couponrate,iscomplicated,available_for_unqualified,sec_type_br_code,original_country,qualified_test_id,qualified_test_id_tm,base_asset_fintoolid,isqualified,listing_level_spb,have_mm_moex,have_mm_spb,beg_book_date,seniority,tradesite,current_coupontype,profit_type,counter,rn
0,2.0,2.0,2.0,2.0,2.0,None,2.0,0.0,2.0,2.0,2.0,2.0,2.0,None,None,None,None,NaN,NaN,None,NaN,NaN,NaN,None,NaN,None,None,None,None,NaN,None,None,None,NaN,NaN,None,None,NaN,NaN,None,None,None,None,None,NaN,None,NaN,NaN,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,S111,None,Старшие необеспеченные,4000,None,None,"МинФин РФ, государственные краткосрочные беску...",None,None,ГКО-21134,NaN,21134RMFS,None,МинФин,0.0,act/365,None,None,Дисконтный,Облигация,None,Необеспеченные,0.0,0.0,0.0,Краткосрочные,None,1.0,RU,0.0,RUB,1000.0,NaN,0.0,0.0,Погашен,2.500000e+09,2500000.0,2.500000e+09,2500000.0,Минфин РФ,00013474,Государственные,6874.0,7710168360,RU,г.Москва,Минфин РФ,00013474,Государственные,6874.0,7710168360,RU,г.Москва,None,0.0,1999-12-17T00:00:00,None,110.0,1999-12-17T00:00:00,2000-04-05T00:00:00,0.0,1999-12-17T00:00:00,Гос,Классические,Постоянный,2023-04-19T15:55:00,NaN,Нет информации,0.0,None,NaN,None,NaN,False,False,False,2.0,NaN,True,None,0.0,False,False,None,RU,1000,1000,NaN,False,NaN,False,False,None,416.0,NaN,None,None,99780,1
1,3.0,3.0,3.0,4.0,3.0,SU21135RMFS7,3.0,1.0,2.0,3.0,2.0,3.0,2.0,Государственные краткосрочные бескупонные обли...,Финансовые органы,None,135,0.0,0.0,None,0.0,0.0,1000.0,RUB,1500000.0,21135RMFS,1999-12-10T00:00:00,Финансовые орган

[SecurityRatingTable](https://docs.efir-net.ru/dh2/#/Rating/SecurityRatingTable?id=post-securityratingtable) - Получить рейтинги нескольких бумаг и связанных с ними компаний на заданную дату.

In [4]:
%%time
SecurityRatingTable = RuDataDF("SecurityRatingTable").df
SecurityRatingTable.head()

SecurityRatingTable started
post start
post start
post start
post start
post start
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
385, 0
post start
post start
post start
post start
post start
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
1731, 385
post start
post start
post start
post start
post start
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
1537, 2116
post start
post start
post start
post start
post start
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
1163, 3653
post start
post start
post start
post start
post start
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
1163, 4816
post start
post start
post start
post start
pos

,id,hid,code_name,id_value,forecast,change,date_award,role,okpo,fininstid,fintoolid,isin,id_rating,code,shortname_rus
0,0,6,MIRA-I-Corp-LT-Nsc,Снят,None,снят,2016-03-18T00:00:00,1,None,7319.0,NaN,None,116,None,Татнефть
1,0,6,MIRA-I-Corp-LT-Nsc,Снят,None,снят,2016-03-18T00:00:00,1,None,43888.0,NaN,None,116,None,Уралкалий
2,0,123,MIRA-I-Cr&D-LT-Nsc,Снят,None,снят,2015-02-25T00:00:00,1,None,43888.0,NaN,None,32,None,Уралкалий
3,0,133,MIRA-B-Cr&D-LT-Nsc,Снят,None,снят,2016-03-18T00:00:00,1,None,7205.0,NaN,None,117,None,Сбербанк России
4,0,152,NRA-I-Cr&D-Nsc,Снят,None,снят,2008-04-09T00:00:00,1,None,75326.0,NaN,None,40,None,Тверьэнерго Фл РосЦентр


[ExchangeTree](https://docs.efir-net.ru/dh2/#/Info/ExchangeTree?id=post-exchangetree) - Получить иерархию торговых площадок/источников, используемых Интерфакс

In [5]:
ExchangeTree = RuDataDF("ExchangeTree").df
ExchangeTree.head()

ExchangeTree started
post start
post start
post start
post start
post start
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
238, 0
post start
post start
post start
post start
post start
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
0, 238
ExchangeTree finished. ExchangeTree shape (238, 10)


,id,id_parent,shortname_rus,shortname_eng,fullname_rus,fullname_eng,sort_order,update_date,counter,rn
0,144,63,Нал,Cash,Наличная валюта,Cash,0,2020-12-28T17:03:09,238,1
1,270,63,RUDIP 2,RUDIP 2,RUDIP 2,RUDIP 2,1,2022-10-31T19:41:24,238,2
2,84,83,США,US,Макроиндикаторы США,USA macroindicators,1,2020-12-28T17:03:09,238,3
3,85,83,РФ,RU,Макроиндикаторы Россия,Russia macroindicators,2,2020-12-28T17:03:09,238,4
4,2,1,РТС,RTS,РТС индексы,RTS indexes,2,2020-12-28T17:03:09,238,5


[ListScaleValues](https://docs.efir-net.ru/dh2/#/Rating/ListScaleValues?id=post-listscalevalues) - Список шкал значений рейтингов

In [6]:
ListScaleValues = RuDataDF("ListScaleValues").df
ListScaleValues.head()

ListScaleValues started
post start
post processing
sleep 1
ListScaleValues finished. ListScaleValues shape (393, 8)


,id,scale_id,rating_value,rating_level,fullname_rus,description,aggregation_type,aggregation_level
0,74,4,AAA,1,Наивысший уровень кредитоспособности,Рейтинги категории «AAA» обозначают самые низк...,BIG3,1
1,75,4,AA+,2,None,None,BIG3,2
2,76,4,AA,3,Очень высокая кредитоспособность,Рейтинги категории «AA» обозначают очень низки...,BIG3,3
3,77,4,AA-,4,None,None,BIG3,4
4,78,4,A+,5,None,None,BIG3,5


[ListRatings](https://docs.efir-net.ru/dh2/#/Dictionary/ListRatings?id=post-listratings) - Список рейтингов

In [7]:
ListRatings = RuDataDF("ListRatings").df
ListRatings.head()

ListRatings started
post start
post processing
sleep 1
ListRatings finished. ListRatings shape (357, 26)


,id,rating_id,code_name,agency,fullname_rus,fullname_eng,for_instrument,for_company,agency_eng,official_name,is_credit,term_type,term_type_name,currency_type,currency_type_name,scale_type,scale_type_name,is_archive,is_accred_br,doc_511p,doc_421p,doc_180i,agency_id,usedInAgg,scale_id,aggregation_type
0,134,104,MDS-B-Cr&D-LT-KZsc,Мудиз,"Рейтинг банковских депозитов, долгосрочный, Ка...","Bank deposit ratings, long-term, kazakhstan scale",0,1,Moody's,LT Bank Deposits (kaz),1,L,Долгосрочный,L,Локальная,N,Национальная,0,1.0,0,0,0,2,False,22,None
1,135,109,MDS-B-Cr&D-LT-UAsc,Мудиз,"Рейтинг банковских депозитов, долгосрочный, Ук...","Bank deposit ratings, long-term, ukrainian scale",0,1,Moody's,LT Bank Deposits (ukr),1,L,Долгосрочный,L,Локальная,N,Национальная,0,1.0,0,0,0,2,False,23,None
2,451,110,RUS-AM-FS&R-Isc,РусРейтинг,"Рейтинг надежности управляющих компаний, между...","Reliability rating of the management company, ...",0,1,RusRating,Рейтинги надежности управляющих компаний,0,None,None,None,None,I,Международная,1,NaN,0,0,0,6,False,35,None
3,452,111,RUS-AM-FS&R-Nsc,РусРейтинг,"Рейтинг надежности управляющих компаний, нацио...","Reliability rating of the management company, ...",0,1,RusRating,Рейтинги надежности управляющих компаний,0,None,None,L,Локальная,N,Национальная,1,NaN,0,0,0,6,False,36,None
4,446,114,EXP-IN-FS&R-Isc,Эксперт РА,"Рейтинги надежности страховых компаний, междун...","Reliability rating of insurance companies, int...",0,1,Expert RA,Рейтинги финансовой надежности страховых компаний,1,None,None,None,None,I,Международная,0,1.0,0,0,0,4,False,47,None


[Emitents](https://docs.efir-net.ru/dh2/#/Info/Emitents?id=post-emitents) - Получить краткий справочник по эмитентам.

In [8]:
Emitents = RuDataDF("Emitents").df
Emitents.head()

Emitents started
post start
post start
post start
post start
post start
post processing
post processing
post processing
sleep 1
post processing
sleep 1
sleep 1
sleep 1
post processing
sleep 1
1500, 0
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
sleep 1
post processing
sleep 1
sleep 1
sleep 1
sleep 1
1500, 1500
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
sleep 1
sleep 1
sleep 1
sleep 1
post processing
sleep 1
1500, 3000
post start
post start
post start
post start
post start
post processing
post processing
post processing
post processing
post processing
sleep 1
sleep 1
sleep 1
sleep 1
sleep 1
1500, 4500
post start
post start
post start
post start
post start
post processing
post processing
post processing
sleep 1
sleep 1
sleep 1
post processing
post processing
sleep 1
sleep 1
1500, 6000
post start
post start
post start
post start
post start
p

,id_emitent,fininstid,shortname_rus,shortname_eng,inn,okpo,ogrn,code,sector,legal_address,phone,www,update_date,fullname_rus_nrd,fullname_eng_nrd,shortname_rus_nrd,shortname_eng_nrd,company_type_short_name,state_reg_num,state_reg_date,state_reg_name,egrul_date,egrul_organ_name,country,lei_code,bik,fax,e_mail,credit_cmp,is_bank_4_non_resident,swift,country_oksm,country_oksm_cbr,country_name_rus,country_name_eng,region_inn,region_soato,region_name,fullname_rus,sparkid,br_fcsm_reg_code,capital,capital_unit,okved,oecd_lvl,oktmo,okato,post_address,okopf,okogu,have_rating,have_risk,is_finorg,is_control_by_finorg,is_gov_or_cb,on_behalf_of_state,is_subjectrf,is_cis_reg,issuer_nrd,tin,kpp,state_reg_number,market_id_nrd,market_name_rus_nrd,market_name_eng_nrd,sp_rx_entity_id,sp_shortname,mds_shortname,mds_org_id,fch_id,fch_shortname,is_monopoly,is_strategic,ifo_list,isregion,sic,sector4212u,sna2008,reg_code,reg_date,reg_org,note,primary_reg_date,inn_fle,other_tin,other_tin_name,okfs_id,okfs_name,is_branch,counter,rn
0,28739,68432,Уралвагонзавод НПК,None,6623029538,7518941,1086623002190,UVZD,Машиностроение,"622007,Свердловская обл., г. Нижний Тагил, шос...",(343)2145232; (3435)403737; (34383)58642,http://http,2024-04-06T15:55:20,"Акционерное общество ""Научно-производственная ...","Joint Stock Company ""Research and production c...","АО ""Научно-производственная корпорация ""Уралва...","JSC ""RPC ""Uralvagonzavod""",АО,1086623002190,2008-03-31T00:00:00,МИФНС России № 16 по Свердловской области,2008-03-31T00:00:00,Межрайонная инспекция Федеральной налоговой сл...,RUS,None,None,8 (3435) 345-067,cb18@uvz.ru,0,0,None,643.0,643.0,РОССИЯ,Russia,66,65,Свердловская область,"""Научно-производственная корпорация ""Уралвагон...",84028.0,55004-F,8.932495e+10,RUB,30.40,7,65751000001,65476370000,"622007,Свердловская обл., г. Нижний Тагил, шос...",12267,4210001,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,MC0324700000,None,662301001,1086623002190,2.0,Корпоративный,corporate,None,None,URALVAGONZAVOD OJSC,823474042,None,None,0,0,None,0.0,3760,None,S111,1086623002190,2008-03-31T00:00:00,Межрайонная инспекция Федеральной налоговой сл...,None,2008-03-31T00:00:00,None,None,ДопИНН,12,Федеральная собственность,False,40516,1201
1,24326,68434,Кировский ЗдСпб,Kirovskij zavod,7805019279,7519047,1027802712365,KIRZ,Машиностроение,"198097, г. Санкт-Петербург, проспект Стачек, д...",(812)3203815; (812)2520416; (812)3026269,www.kzgroup.ru/ru/invest,2024-04-06T15:52:50,"Публичное акционерное общество ""Кировский завод""","Public Joint-Stock Company ""Kirovsky Zavod""","ПАО ""Кировский завод""","PJSC ""Kirovsky Zavod""",ПАО,1473,1992-11-05T00:00:00,Регистрационная палата мэрии Санкт-Петербурга,2002-07-16T00:00:00,Инспекция Министерства Российской Федерации по...,RUS,None,None,+7 (812) 3203812,galina.klinduh@kzgroup.ru,0,0,None,643.0,643.0,РОССИЯ,Russia,78,40,г.Санкт-Петербург,"""Кировский завод"", ПАО",84031.0,00046-A,1.087000e+06,RUB,68.20.2,7,40339000000,40276000000,"198097, г. Санкт-Петербург, проспект Стачек, д...",12247,4210008,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,MS0220300000,None,780501001,1473,2.0,Корпоративный,corporate,None,None,None,None,None,None,0,0,None,0.0,6510,None,S112,1027802712365,2002-07-16T00:00:00,Регистрационная палата Мэрии Санкт-Петербурга,None,1992-11-05T00:00:00,None,None,ДопИНН,16,Частная собственность,False,40516,1202
2,25863,68437,Волгоградский СудострЗд,Volgogradskij sudostroitelnyj zavod,3448006530,7519550,1023404355270,VGSZ,Машиностроение,"400112, Волгоградская обл., г. Волгоград, ул. ...",(844)2612535; (844)2491949; (844)2612547,www.vgsz.ru,2024-04-06T15:52:50,"Открытое акционерное общество ""Волгоградский с...",None,"ОАО ""ВгСЗ""",None,ОАО,1,1992-12-25T00:00:00,Администрация Красноармейского района г. Волго...,2002-07-12T00:00:00,Инспекция Министерства РФ по налогам и сборам ...,RUS,None,None,(8442) 67-43-89,kvn@vgsz.ru,0,0,None,643.0,643.0,РОССИЯ,Russia,34,18,Волгоградская область,"""Волгоградский судостроительный завод"", ОАО",1117985.0,45440-E,1.34122

[OfferorsGuarants](https://docs.efir-net.ru/dh2/#/Bond/OfferorsGuarants?id=post-offerorsguarants) - Возвращает список гарантов/оферентов для инструмента

In [9]:
OfferorsGuarants = RuDataDF("OfferorsGuarants").df
OfferorsGuarants.head()

OfferorsGuarants started
post start
post start
post start
post start
post start
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
500, 0
post start
post start
post start
post start
post start
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
500, 500
post start
post start
post start
post start
post start
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
500, 1000
post start
post start
post start
post start
post start
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
500, 1500
post start
post start
post start
post start
post start
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
500, 2000
post start
post start
post start
post start
post start

,fintoolId,fininstid_guarant,fininstid_offeror,counter,rn
0,135564,87166,NaN,5183,401
1,135601,95739,NaN,5183,402
2,135627,87166,NaN,5183,403
3,135628,87166,NaN,5183,404
4,135645,106640,NaN,5183,405


[CompanyRatingsTable](https://docs.efir-net.ru/dh2/#/Rating/CompanyRatingsTable?id=post-companyratingstable) - Получить рейтинги нескольких компаний на заданную дату.

In [10]:
CompanyRatingsTable = RuDataDF("CompanyRatingsTable").df
CompanyRatingsTable.head()

CompanyRatingsTable started
post start
post start
post start
post start
post start
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
760, 0
post start
post start
post start
post start
post start
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
187, 760
post start
post start
post start
post start
post start
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
733, 947
post start
post start
post start
post start
post start
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
757, 1680
post start
post start
post start
post start
post start
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
post processing
sleep 1
559, 2437
post start
post start
post start
post start
post sta

,name_ra,dt,last,last_dt,change,forecast,advanced,prev,prev_dt,id_rating,code,code_type,company_name
0,Рейтинг отозван,2008-10-02T00:00:00,Снят,2008-10-02T00:00:00,снят,None,Основной,A2.ru,2008-05-06T00:00:00,6,7482,FININSTID,ЧТПЗ
1,"Рейтинг подтвержден, прогноз Позитивный",2024-04-03T00:00:00,a(ru),2017-05-19T00:00:00,Подтвержден,Позитивный,Основной,None,None,200,7463,FININSTID,ТБанк
2,"Рейтинг повышен, прогноз Стабильный",2023-10-19T00:00:00,aa-(ru),2023-10-19T00:00:00,Повышен,Стабильный,Основной,a+(ru),2022-11-09T00:00:00,200,7534,FININSTID,Эльгауголь
3,Рейтинг подтвержден,2024-03-18T00:00:00,a(ru),2020-04-30T00:00:00,Подтвержден,Стабильный,Основной,None,None,200,7574,FININSTID,ЯТЭК
4,"Рейтинг подтвержден, прогноз Стабильный",2023-11-20T00:00:00,a+(ru),2021-11-30T00:00:00,Подтвержден,Стабильный,Основной,a(ru),2020-11-30T00:00:00,200,7623,FININSTID,КуйбышевАзот


#### Шаг 2. Предобработка таблиц до экспорта в excel

In [11]:
#добавляем информацию из справочника в нашу таблицу с компаниями
ExtendedSecurityRatingTable = SecurityRatingTable.merge(ListRatings[['rating_id','agency','official_name','term_type_name','scale_type_name','scale_id']], how='left', left_on='id_rating', right_on='rating_id')
ExtendedSecurityRatingTable = ExtendedSecurityRatingTable.merge(ListScaleValues, how='left', left_on=['scale_id', 'id_value'], right_on=['scale_id', 'rating_value'])
ExtendedSecurityRatingTable = ExtendedSecurityRatingTable[[
    'id_x', 'hid', 'code_name', 'id_value', 'forecast', 'change',
    'date_award', 'role', 'okpo', 'fininstid', 'fintoolid', 'isin',
    'id_rating', 'code', 'shortname_rus', 'rating_id', 'agency',
    'official_name', 'term_type_name', 'scale_type_name', 'scale_id',
    'aggregation_level', 'aggregation_type']].rename(
    columns={
    'id_x': 'id', 'aggregation_level': 'level', 'aggregation_type': 'agg_type'
    }
)
agencies = ['АКРА', 'Эксперт РА', 'НКР', 'НРА'
           # ]
            , 'Мудис Интерфакс']
ExtendedSecurityRatingTable = ExtendedSecurityRatingTable[ExtendedSecurityRatingTable['agency'].isin(agencies)]
ExtendedSecurityRatingTable = ExtendedSecurityRatingTable[ExtendedSecurityRatingTable['term_type_name'] != 'Краткосрочный']
ExtendedSecurityRatingTable = ExtendedSecurityRatingTable[~ExtendedSecurityRatingTable['id_value'].isin(['****','Приостановлен','Снят'])]
ExtendedSecurityRatingTable = ExtendedSecurityRatingTable[~ExtendedSecurityRatingTable['isin'].isna()]


In [12]:
akra_list = [
'Structured Finance Instrument Rating',
 'Рейтинг эмиссии, международная шкала',
 'Старший необеспеченный рейтинг эмиссии',
 'Старший обеспеченный рейтинг эмиссии',
 'Субординированный рейтинг эмиссии'
]
expert_list = [
    'Structured finance instrument rating',
    'Кредитный рейтинг долговых инструментов'
]
nkr_list = [
   # 'Кредитный рейтинг долговых обязательств'
]
nra_list = [
 #   'Кредитный рейтинг отдельных выпусков облигаций',
  #  'Рейтинг кредитоспособности выпусков облигаций Индивидуальный'
]
moodys_list = [
    'NSR Senior Unsecured (Domestic)',
 'Senior Unsecured (Domestic); Long-Term Corporate Obligation Rating (Domestic); Structured Finance Long-Term Rating (Domestic)',
 'Senior Unsecured (Foreign); Long-Term Corporate Obligation Rating (Foreign); Structured Finance Long-Term Rating (Foreign)',
 'Subordinate (Domestic)',
 'Subordinate (Foreign)'
]

ExtendedSecurityRatingTable = ExtendedSecurityRatingTable[
((ExtendedSecurityRatingTable['agency']=='АКРА') & (ExtendedSecurityRatingTable['official_name'].isin(akra_list))) |
((ExtendedSecurityRatingTable['agency']=='Эксперт РА') & (ExtendedSecurityRatingTable['official_name'].isin(expert_list))) |
((ExtendedSecurityRatingTable['agency']=='НКР') & (ExtendedSecurityRatingTable['official_name'].isin(nkr_list))) |
((ExtendedSecurityRatingTable['agency']=='НРА') & (ExtendedSecurityRatingTable['official_name'].isin(nra_list))) |
((ExtendedSecurityRatingTable['agency']=='Мудис Интерфакс') & (ExtendedSecurityRatingTable['official_name'].isin(moodys_list)))
 ]

ExtendedSecurityRatingTable = ExtendedSecurityRatingTable\
.dropna(subset=['isin'])\
.drop_duplicates(subset=["isin", "agency"])\
.reset_index(drop=True)

In [13]:
ExtendedSecurityRatingTablePivot = ExtendedSecurityRatingTable.drop_duplicates().pivot(index='isin', columns='agency', values='id_value')
ExtendedSecurityRatingTablePivot.columns = pd.MultiIndex.from_tuples([('Эмиссия', i) for i in ExtendedSecurityRatingTablePivot.columns])

In [14]:
TransformedFintoolReferenceData = FintoolReferenceData.copy()[['isincode', 'nickname', 'fintooltype', 'country', 'faceftname', 'fintoolid',
                   'issuername', 'issuercountry', 'issuerinn', 'issueruid',
                   'borrowername', 'borrowercountry', 'borrowerinn', 'borroweruid',
                   'isguaranteed', 'guaranteetype', 'guaranteeamount', 'guarantval',
                   ]].drop_duplicates(subset=['isincode'])
TransformedFintoolReferenceData[['fintoolid', 'issueruid', 'borroweruid']] = TransformedFintoolReferenceData[['fintoolid', 'issueruid', 'borroweruid']].fillna(0).astype('int')
TransformedFintoolReferenceData = TransformedFintoolReferenceData.merge(OfferorsGuarants[['fintoolId','fininstid_guarant']], how='left', left_on='fintoolid', right_on='fintoolId')
ids = list(TransformedFintoolReferenceData.loc[~TransformedFintoolReferenceData['fininstid_guarant'].isna(),'fininstid_guarant'].astype('int'))
TransformedEmitents = Emitents[Emitents.fininstid.isin(ids)][['shortname_rus','country','inn','fininstid']]
TransformedEmitents.columns = ['name_guarant','country_guarant','inn_guarant','fininstid']
TransformedFintoolReferenceData = TransformedFintoolReferenceData.merge(TransformedEmitents, how='left', left_on='fininstid_guarant', right_on='fininstid')
TransformedFintoolReferenceData = TransformedFintoolReferenceData.drop(columns=['fintoolId', 'fininstid'])
TransformedFintoolReferenceData = TransformedFintoolReferenceData.set_index('isincode').rename_axis('ISIN')
TransformedFintoolReferenceData = TransformedFintoolReferenceData.rename(
    columns={
        'nickname': ('Эмиссия','Название'),
        'fintooltype': ('Эмиссия','Тип'),
        'country': ('Эмиссия','Страна'), 
        'faceftname': ('Эмиссия','Валюта'), 
        'fintoolid': ('Эмиссия','Id RuData'),
        'issuername': ('Эмитент','Название'), 
        'issuercountry': ('Эмитент','Страна'), 
        'issuerinn': ('Эмитент','ИНН'), 
        'issueruid': ('Эмитент','Id RuData'), 
        'borrowername': ('Заемщик','Название'),
        'borrowercountry': ('Заемщик','Страна'), 
        'borrowerinn': ('Заемщик','ИНН'), 
        'borroweruid': ('Заемщик','Id RuData'), 
        'isguaranteed': ('Гарант','Есть гарантия'),
        'guaranteetype': ('Гарант','Вид гарантии'), 
        'guaranteeamount': ('Гарант','Полнота гарантии'), 
        'guarantval': ('Гарант','Гарантированная сумма'), 
        'fininstid_guarant': ('Гарант','Id RuData'),
        'name_guarant': ('Гарант','Название'), 
        'country_guarant': ('Гарант','Страна'), 
        'inn_guarant': ('Гарант','ИНН')
    }
)
TransformedFintoolReferenceData.columns = pd.MultiIndex.from_tuples(TransformedFintoolReferenceData.columns)

In [15]:
ExtendedFintoolReferenceData = TransformedFintoolReferenceData.join(ExtendedSecurityRatingTablePivot)
new_cols = ExtendedFintoolReferenceData.columns.reindex(['Эмиссия','Эмитент','Заемщик','Гарант'], level=0)
ExtendedFintoolReferenceData = ExtendedFintoolReferenceData.reindex(columns=new_cols[0])

In [16]:
ExtendedCompanyRatingsTable = CompanyRatingsTable.merge(ListRatings[['id','agency','official_name','term_type_name','scale_type_name','scale_id']], how='left', left_on='id_rating', right_on='id')
##добавляем уровень рейтинг
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable.merge(ListScaleValues, how='left', left_on=['scale_id', 'last'], right_on=['scale_id', 'rating_value'])

In [17]:
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable[['name_ra', 'dt', 'last', 'last_dt', 'change', 'forecast', 'advanced',
           'prev', 'prev_dt', 'id_rating', 'code', 'code_type', 'company_name',
           'id_x', 'agency', 'official_name', 'term_type_name', 'scale_type_name',
           'scale_id', 'aggregation_level', 'aggregation_type']].rename(columns={'id_x': 'id', 'aggregation_level': 'level', 'aggregation_type': 'agg_type'})
ExtendedCompanyRatingsTable['id_rating'] = ExtendedCompanyRatingsTable['id_rating'].fillna(0)

In [18]:
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable[ExtendedCompanyRatingsTable['agency'].isin(['АКРА','Эксперт РА','НКР','НРА','Мудис Интерфакс'])]
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable[~ExtendedCompanyRatingsTable['last'].isin(['****','Приостановлен','Снят'])]
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable[ExtendedCompanyRatingsTable['term_type_name'] != 'Краткосрочный']

In [19]:
akra_list=[
 'Рейтинг эмитентов',
 'Long-term credit rating',
 'Long-term local currency credit rating',
 'Long-term foreign currency credit rating',
]
expert_list=[
 'Рейтинг кредитоспособности организаций, осуществляющих микрофинансовую деятельность (Республика Казахстан)',
 'Рейтинг кредитоспособности проектных компаний',
 'Рейтинги кредитоспособности банка Республики Беларусь',
 'Рейтинги кредитоспособности банков',
 'Рейтинги кредитоспособности депозитариев',
 'Рейтинги кредитоспособности лизинговых компаний',
 'Рейтинги кредитоспособности микрофинансовых организаций',
 'Рейтинги кредитоспособности нефинансовых компаний',
 'Рейтинги кредитоспособности нефинансовых компаний Республики Беларусь',
 'Рейтинги кредитоспособности нефинансовых компаний Республики Казахстан',
 'Рейтинги кредитоспособности субъектов федерации и муниципальных образований',
 'Рейтинги кредитоспособности факторинговых компаний',
 'Рейтинги кредитоспособности финансовых компаний',
 'Рейтинги кредитоспособности холдинговых компаний',
 'Рейтинги надежности компаний по страхованию жизни',
 'Рейтинги финансовой надежности негосударственных пенсионных фондов',
 'Рейтинги финансовой надежности страховых компаний'
]
nkr_list=[
 'Кредитный рейтинг НКР по Национальной шкале, Банки',
 'Кредитный рейтинг НКР по Национальной шкале, Нефинансовые компании'
]
nra_list=[
 'Кредитный рейтинг кредитных организаций',
 'Кредитный рейтинг нефинансовых компаний, national scale',
]
moodys_list=[
 'BACKED LT Bank Deposits',
 'BACKED LT Bank Deposits (Domestic)',
 'BACKED LT Bank Deposits (Foreign)',
 'BACKED LT Counterparty Risk Rating',
 'BACKED LT Counterparty Risk Rating (Domestic)',
 'BACKED LT Counterparty Risk Rating (Foreign)',
 'BACKED LT Issuer Rating',
 'BACKED LT Issuer Rating (Domestic)',
 'BACKED LT Issuer Rating (Foreign)',
 'BACKED Senior Unsecured (Domestic)',
 'BACKED Senior Unsecured (Foreign)',
 'Issuer Ratings',
 'LT Bank Deposits',
 'LT Bank Deposits (Domestic)',
 'LT Bank Deposits (Foreign)',
 'LT Bank Deposits (kaz)',
 'LT Bank Deposits (ukr)',
 'LT Corporate Family Ratings',
 'LT Corporate Family Ratings (Domestic)',
 'LT Corporate Family Ratings (Foreign)',
 'LT Corporate Family Ratings (ukr)',
 'LT Counterparty Risk Rating (Domestic)',
 'LT Counterparty Risk Rating (Foreign)',
 'LT Insurance Financial Strength',
 'LT Insurance Financial Strength (Domestic)',
 'LT Insurance Financial Strength (Foreign)',
 'LT Issuer Rating',
 'LT Issuer Rating (Domestic)',
 'LT Issuer Rating (Foreign)',
 'Senior Unsecured',
 'Senior Unsecured (Domestic)',
 'Senior Unsecured (Foreign)',
]

ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable[
((ExtendedCompanyRatingsTable['agency']=='АКРА') & (ExtendedCompanyRatingsTable['official_name'].isin(akra_list))) |
((ExtendedCompanyRatingsTable['agency']=='Эксперт РА') & (ExtendedCompanyRatingsTable['official_name'].isin(expert_list))) |
((ExtendedCompanyRatingsTable['agency']=='НКР') & (ExtendedCompanyRatingsTable['official_name'].isin(nkr_list))) |
((ExtendedCompanyRatingsTable['agency']=='НРА') & (ExtendedCompanyRatingsTable['official_name'].isin(nra_list))) |
((ExtendedCompanyRatingsTable['agency']=='Мудис Интерфакс') & (ExtendedCompanyRatingsTable['official_name'].isin(moodys_list)))
]

In [20]:
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable.dropna(subset=['code'])
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable.reset_index(drop=True)

In [21]:
for n, g in ExtendedCompanyRatingsTable.groupby(['code', 'agency']):
    if g.shape[0] > 1: 
        try:
            m = g['level'].idxmin(skipna=False)
            if pd.isna(m):
                m = g.index[0]
            to_drop = g.index.drop(m)
            ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable.drop(index=to_drop)
        except:
            print(g)
            raise
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable.dropna(subset=['code'])
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable.reset_index(drop=True)
ExtendedCompanyRatingsTable['code'] = ExtendedCompanyRatingsTable['code'].astype(int)

In [22]:
ExtendedCompanyRatingsPivot = ExtendedCompanyRatingsTable.pivot(index='code', columns='agency', values='last')

In [23]:
df1 = ExtendedFintoolReferenceData.copy()
df1.columns =df1.columns.to_flat_index()
for k in ['Эмитент','Заемщик','Гарант']:
    df2 = ExtendedCompanyRatingsPivot.copy()
    df2.columns = [(k, i) for i in df2.columns]
    df1 = df1.merge(df2, how='left', left_on=[(k, 'Id RuData')], right_index=True)
df1.columns = pd.MultiIndex.from_tuples(df1.columns)
new_cols = df1.columns.reindex(['Эмиссия', 'Эмитент', 'Заемщик', 'Гарант'], level=0)
df1 = df1.reindex(columns=new_cols[0])
ExtendedFintoolReferenceData = df1

In [51]:
df = Emitents[Emitents['fininstid'].isin(ExtendedCompanyRatingsPivot.index)]
CompanyRatings = df.merge(ExtendedCompanyRatingsPivot, left_on='fininstid', right_index=True, how='left')

#### Шаг 3. Рейтинги облигаций из фактовой таблицы бумаг FintoolReferenceData. Рейтинги облигаций из фактовой таблицы компаний CompanyRatingsTable

In [52]:
BondRatings = ExtendedFintoolReferenceData['Эмиссия'].reset_index()
id_vars = BondRatings.columns[:6]
value_vars = BondRatings.columns[6:]
meltBonds = pd.melt(BondRatings, id_vars=id_vars, value_vars=value_vars, var_name='agency', value_name='rate')
meltBonds = meltBonds[~meltBonds.rate.isna()].sort_values('Id RuData')
meltBonds = meltBonds[meltBonds["Тип"] == 'Облигация']

In [53]:
CompanyRatings = pd.concat([
    CompanyRatings[["id_emitent", "shortname_rus", "inn", "update_date", "country_oksm", "have_rating", "have_risk", "tin", "fininstid", "reg_code", "lei_code"]]
    , CompanyRatings.iloc[:, CompanyRatings.columns.get_loc('rn')+1:]
], axis=1)
id_vars = CompanyRatings.columns[:11]
value_vars = CompanyRatings.columns[11:]
meltCompanies = pd.melt(CompanyRatings, id_vars=id_vars, value_vars=value_vars, var_name='agency', value_name='rate')
meltCompanies = meltCompanies[meltCompanies['rate'].notnull()].sort_values('id_emitent')
meltCompanies['inn'] = np.where(meltCompanies['country_oksm'] == 643, meltCompanies['inn'], meltCompanies['tin'] )
meltCompanies['country_oksm'] = meltCompanies['country_oksm'].astype(np.int32)
meltCompanies['type'] = " ЮЛ"

In [54]:
meltBonds.rename(columns={
    "Id RuData": "id_emitent",
    "Название": "company_name", 
    "Тип": "rate_type",
    "ISIN": "inn/isin",
    "Страна": "country"
    }, inplace=True)
meltCompanies.rename(columns={
    "id_rudata": "id_emitent",
    "shortname_rus": "company_name", 
    "type": "rate_type",
    "inn": "inn/isin",
    "country_oksm": "country"
    }, inplace=True)
meltBonds["fininstid"] = np.nan
meltBonds["reg_code"] = np.nan
meltBonds["lei_code"] = np.nan
meltBonds["report_date"] = pd.to_datetime(REPORT_DATE)
meltCompanies["report_date"] = pd.to_datetime(REPORT_DATE)

Справочник кодов стран ОКСМ

In [55]:
countries = pd.read_csv("./data/Input/countries.csv", header=0, encoding='cp1251', sep=';')[["country", "alpha2"]]
countries["country"] = countries["country"].astype(np.int32)
co = countries.set_index("alpha2").country.to_dict()
meltBonds["country_code"] = meltBonds["country"].map(co)
meltCompanies["country_code"] = meltCompanies["country"].map(co)

#### Шаг 4. Получаем результирующий результат по рейтингам облигаций и компаний и экспорт в excel

In [58]:
meltBonds = meltBonds[["id_emitent", "fininstid", "company_name", "inn/isin", "country_code", "agency", "rate", "report_date", "rate_type", "reg_code", "lei_code"]]
meltCompanies = meltCompanies[["id_emitent", "fininstid", "company_name", "inn/isin", "country_code", "agency", "rate", "report_date", "rate_type", "reg_code", "lei_code"]]
ResultRatings = pd.concat([meltBonds, meltCompanies], axis=0)
with pd.ExcelWriter('data/Output/ratings.xlsx', datetime_format='dd.mm.yyyy') as writer:
    ResultRatings.to_excel(writer, index=False)